In [ ]:
#installing libraries
!pip install transformers --quiet
!sudo python -m pip install html2text
!pip install wikipedia
!pip install flask_wtf
!pip install flask-ngrok
!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 3.1 MB 12.7 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
     |████████████████████████████████| 3.3 MB 37.7 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=13f8f45bf015425135cab9a89241bc3ee7feee960b64899525303732a149b90b
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
     |████████████████████████████████| 136 kB 15.5 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.3 MB 39.1 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl 

In [ ]:
#getting flask app files
!unzip '/content/QA_app.zip'

Archive:  /content/QA_app.zip
   creating: QA_app/
  inflating: __MACOSX/._QA_app       
  inflating: QA_app/.DS_Store        
  inflating: __MACOSX/QA_app/._.DS_Store  
   creating: QA_app/models/
  inflating: __MACOSX/QA_app/._models  
   creating: QA_app/__pycache__/
  inflating: __MACOSX/QA_app/.___pycache__  
   creating: QA_app/static/
  inflating: __MACOSX/QA_app/._static  
  inflating: QA_app/app.py           
  inflating: __MACOSX/QA_app/._app.py  
   creating: QA_app/templates/
  inflating: __MACOSX/QA_app/._templates  
  inflating: QA_app/models/.DS_Store  
  inflating: __MACOSX/QA_app/models/._.DS_Store  
   creating: QA_app/models/trained_model/
  inflating: __MACOSX/QA_app/models/._trained_model  
   creating: QA_app/models/__pycache__/
  inflating: __MACOSX/QA_app/models/.___pycache__  
  inflating: QA_app/models/model.py  
  inflating: __MACOSX/QA_app/models/._model.py  
  inflating: QA_app/models/gradcam.py  
  inflating: __MACOSX/QA_app/models/._gradcam.py  
  inflati

In [ ]:
#importing libraries
import numpy as np
import requests
import html2text
from googlesearch import search
import json
import re
from bs4 import BeautifulSoup
from markdown import markdown
import wikipedia
from difflib import SequenceMatcher

In [ ]:
def markdown_to_text(markdown_string):
    # md -> html -> text 
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ""
    for para in soup.find_all('p'): #getting all the paragraph elements for context
      data = para.get_text()
      if(len(data.split()) > 20): #only include paragraphs with more than 20 words
        text += data
    return text

def format_text(text):
    text = markdown_to_text(text)
    text = text.replace('\n', ' ') #remove endline character
    return text

In [ ]:
def query_pages(query, n=5):
    return list(search(query, num=n, stop=n, pause=2))

def query_to_text(query, n=5):
    html_conv = html2text.HTML2Text()
    html_conv.ignore_links = True
    html_conv.escape_all = True
    
    text = []
    links = query_pages(query, n)
    session = requests.Session()
    for link in links:
        req = session.get(link)
        print(link)
        text.append(html_conv.handle(req.text))
        text[-1] = format_text(text[-1])
        if text[-1] == "": #remove if no paragraphs were found
          text.pop()

    return text

In [ ]:
query_to_text('What is the color of the sky?',10)

https://www.universetoday.com/74020/what-color-is-the-sky/
https://spaceplace.nasa.gov/blue-sky/en/
https://science.discoveryplace.org/blog/ever-wonder-why-some-animals-see-a-purple-sky
https://medium.com/the-shadow/the-colours-of-the-sky-20ee98e6a9de
https://scijinks.gov/blue-sky/
http://www.webexhibits.org/causesofcolor/14.html
https://www.encyclopedie-environnement.org/en/air-en/colours-sky/
https://www.quora.com/What-is-the-actual-colour-of-the-sky
https://www.quora.com/Is-the-colour-of-the-sky-actually-blue-or-is-it-a-mere-illusion
https://www.quora.com/What-colour-is-the-night-sky


['Sunlight reaches Earth\'s atmosphere and is scattered in all directions by all the gases and particles in the air. Blue light is scattered more than the other colors because it travels as shorter, smaller waves. This is why we see a blue sky most of the time.When white light shines through a prism, the light is separated into all its colors. A prism is a specially shaped crystal.If you visited The Land of the Magic Windows, you learned that the light you see is just one tiny bit of all the kinds of light energy beaming around the universe--and around you!Like energy passing through the ocean, light energy travels in waves, too. Some light travels in short, "choppy" waves. Other light travels in long, lazy waves. Blue light waves are shorter than red light waves.Sunlight reaches Earth\'s atmosphere and is scattered in all directions by all the gases and particles in the air. Blue light is scattered in all directions by the tiny molecules of air in Earth\'s atmosphere. Blue is scattere

In [ ]:
def get_wikipedia_context(query,n):
  text = []
  topics = wikipedia.search(query, results = n)
  for topic in topics:
    try:
      print(topic)
      text.append(wikipedia.page(topic).content)
    except:
      pass
  return text

In [ ]:
def split_contexts(context, seq_len, top):
  split_context = []
  sentences = context.split('.')

  for s in sentences:
    s += '.' # add the full stop back.

  short = []
  for s in sentences:
    if len(s) < seq_len: #only consider sentences which are less than the sequence length 
      short.append(s)

  i = 0
  filled = 0
  temp = ""
  while(i < len(short) and top > 0):
    next = short[i]
    if (len(next) + filled > seq_len):
      split_context.append(temp)
      temp = ""
      filled = 0
      top -= 1
    else:
      filled += len(next)
      temp += (next + '. ')
      i+=1
  
  if temp != "" and top > 0:
    split_context.append(temp)
    
  return split_context

In [ ]:
from googletrans import Translator
languages = ['fr','de','zh-cn','es','ja']
def generate_questions(question):
  variants = [question.lower()]
  translator = Translator()
  for l in languages:
    en_to_l = translator.translate(question, dest=l) #english to x
    l_to_en = translator.translate(en_to_l.text, dest='en') #x to english
    translated = l_to_en.text.lower()
    if translated not in variants:
      variants.append(translated)
  print('Variants:',variants)
  return variants

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

def create_model():
    model_name = "deepset/roberta-base-squad2"
    return pipeline('question-answering', model=model_name, tokenizer=model_name)
    
def predict_answer(model, question, contexts, top ,seq_len=512, debug=False):
    split_context = []
    
    if not isinstance(contexts, list):
        contexts = [contexts]
    print('Spliting Contexts...')
    for context in contexts:
      split_context.extend(split_contexts(context, seq_len, top))

    f_data = []
    print('Generating Question Variants...')
    questions = generate_questions(question)

    for i, c in enumerate(split_context):
      for q in questions:
        f_data.append(
            {
              'question': q,
              'context': c
            })
        
    preds = []
    print('Predicting Answers...')
    prediction = model(f_data)

    if not isinstance(prediction, list):
        prediction = [prediction]

    for p in prediction:
      if p['score'] > 0.50:
        preds.append(p['answer'].lower().strip())

    if len(preds) == 0: #if not confident make best prediction
      for p in prediction:
        preds.append(p['answer'].lower().strip())

    print(preds)

    return max(set(preds), key = preds.count)

In [ ]:
def q_to_a(model, question, choice, context, n=2, debug=False):
    if choice == 'Google':
      context = query_to_text(question, n=n)
    elif choice == 'Wikipedia':
      context = get_wikipedia_context(question, n)
    pred = predict_answer(model, question, context, 2,debug=debug)
    return pred

In [ ]:
model = create_model()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from flask import Flask,render_template,flash, redirect, url_for, request
from flask_wtf import FlaskForm
from wtforms import StringField, TextAreaField, SubmitField, SelectField
from wtforms.validators import DataRequired,Length
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)  

app.config['SECRET_KEY'] = '1de2d95051ff6e819cdb7c62c92bdd5f'

class Form(FlaskForm):
    question = StringField('Question', validators=[DataRequired(),Length(min=10)])
    choice = SelectField('Choice', choices = ['Custom','Wikipedia','Google'])
    context = TextAreaField('Context', render_kw={"rows": 10, "cols": 11})
    submit = SubmitField('Submit')


@app.route("/", methods=['GET','POST'])
def home():
    form = Form()
    if form.validate_on_submit():
        question = form.question.data
        context = form.context.data
        choice = form.choice.data
        return redirect(url_for('prediction',question = question, context=context, choice=choice))
    return render_template('home.html',form=form)
    
@app.route("/prediction")
def prediction():
    question = request.args['question']
    context = request.args['context']
    choice = request.args['choice']
    answer = q_to_a(model, question, choice, context, n=10)
    print('Answer:', answer)
    return render_template('prediction.html', answer = answer)

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5e95-34-91-27-96.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Dec/2021 09:20:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 09:20:48] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 09:20:49] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Dec/2021 09:21:04] "POST / HTTP/1.1" 302 -


https://www.universetoday.com/74020/what-color-is-the-sky/
https://spaceplace.nasa.gov/blue-sky/en/
https://science.discoveryplace.org/blog/ever-wonder-why-some-animals-see-a-purple-sky
https://medium.com/the-shadow/the-colours-of-the-sky-20ee98e6a9de
https://scijinks.gov/blue-sky/
http://www.webexhibits.org/causesofcolor/14.html
https://www.encyclopedie-environnement.org/en/air-en/colours-sky/
https://www.quora.com/What-is-the-actual-colour-of-the-sky
https://www.quora.com/Is-the-colour-of-the-sky-actually-blue-or-is-it-a-mere-illusion
https://www.quora.com/What-colour-is-the-night-sky
Spliting Contexts...
Generating Question Variants...
Variants: ['what is the color of the sky?', 'what color is the sky?', 'what is the color of heaven?', 'what is the sky color?']
Predicting Answers...


...

127.0.0.1 - - [02/Dec/2021 09:23:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2021 09:23:32] "POST / HTTP/1.1" 302 -


Spliting Contexts...
Generating Question Variants...
Variants: ['how many strings does a guitar have?', 'how many strings has a guitar?', 'how many strings are guitar?', 'how many ropes does a guitar have?', 'how many guitars do you have?']
Predicting Answers...


127.0.0.1 - - [02/Dec/2021 09:23:37] "GET /prediction?question=How+many+strings+does+a+guitar+have%3F&context=The guitar is+a fretted musical+instrument+with+strings+that+you+play+with+your+fingers+or+with+a+piece+of+plastic+%28a+plectrum%29.+It+typically+has+six strings.+It+is+held+flat+against+the+player%27s+body+and+played+by strumming or plucking the+strings+with+the+dominant+hand%2C+while+simultaneously+pressing+selected+strings+against frets with+the+fingers+of+the+opposite+hand.+A plectrum or+individual+finger picks may+be+used+to+strike+the+strings.+The+sound+of+the+guitar+is+projected+either+acoustically%2C+by+means+of+a+resonant+chamber+on+the+instrument%2C+or+amplified+by+an+electronic pickup and+an amplifier.&choice=Custom HTTP/1.1" 200 -


['six', 'six', 'six']
Answer: six


127.0.0.1 - - [02/Dec/2021 09:26:03] "GET / HTTP/1.1" 200 -
